In [15]:
#
import numpy as np
import os, sys

sys.path.insert(0, '/global/homes/q/qmxp55/DESI/bgstargets/py')

from io_ import get_sweep_whole, getBGSbits
from io_ import get_random, get_isdesi, get_dict, bgsmask, get_reg
from cuts import getGeoCuts, get_bgs, flux_to_mag
from QA import getStats, flow

sys.path.insert(0, '/global/homes/q/qmxp55/DESI/bgs_main/')
from desitarget.cuts import select_targets
from desitarget import io, cuts
from desitarget.sv1 import sv1_cuts

import healpy as hp
import astropy.io.fits as fits
from astropy.coordinates import SkyCoord
import astropy.units as units
import fitsio
import matplotlib.pyplot as plt
from astropy.table import Table

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

def runtest(rff=False, targtype="bright", primary1=False, file=None, survey='main'):
    
    if file is None: raise ValueError('Select a file from SWEEPS, TRACTOR or DESITARGET-like')
    targets = Table.read(file)
    flux = cuts.unextinct_fluxes(targets)
    gflux = flux['GFLUX']
    rflux = flux['RFLUX']
    zflux = flux['ZFLUX']
    w1flux = flux['W1FLUX']
    w2flux = flux['W2FLUX']
    zfiberflux = flux['ZFIBERFLUX']
    rfiberflux = flux['RFIBERFLUX']
    objtype = targets['TYPE']

    gfluxivar = targets['FLUX_IVAR_G']
    rfluxivar = targets['FLUX_IVAR_R']
    zfluxivar = targets['FLUX_IVAR_Z']

    gsnr = targets['FLUX_G'] * np.sqrt(targets['FLUX_IVAR_G'])
    rsnr = targets['FLUX_R'] * np.sqrt(targets['FLUX_IVAR_R'])
    zsnr = targets['FLUX_Z'] * np.sqrt(targets['FLUX_IVAR_Z'])
    w1snr = targets['FLUX_W1'] * np.sqrt(targets['FLUX_IVAR_W1'])
    w2snr = targets['FLUX_W2'] * np.sqrt(targets['FLUX_IVAR_W2'])

    dchisq = targets['DCHISQ']
    deltaChi2 = dchisq[..., 0] - dchisq[..., 1]

    gnobs, rnobs, znobs = targets['NOBS_G'], targets['NOBS_R'], targets['NOBS_Z']
    gallmask = targets['ALLMASK_G']
    rallmask = targets['ALLMASK_R']
    zallmask = targets['ALLMASK_Z']
    gfracflux = targets['FRACFLUX_G']
    rfracflux = targets['FRACFLUX_R']
    zfracflux = targets['FRACFLUX_Z']
    gfracmasked = targets['FRACMASKED_G']
    rfracmasked = targets['FRACMASKED_R']
    zfracmasked = targets['FRACMASKED_Z']
    gfracin = targets['FRACIN_G']
    rfracin = targets['FRACIN_R']
    zfracin = targets['FRACIN_Z']
    maskbits = targets['MASKBITS']
    refcat = targets['REF_CAT']

    gaiagmag = targets['GAIA_PHOT_G_MEAN_MAG']
    Grr = gaiagmag - 22.5 + 2.5*np.log10(targets['FLUX_R'])

    if 'BRICK_PRIMARY' in targets.colnames:
        primary = targets['BRICK_PRIMARY']
    else:
        primary = np.ones_like(gflux, dtype='?')
    
    if rff: ff = rfiberflux
    else: ff = None
    
    if primary1: prim = primary
    else: prim = None
    
    if survey == 'main':
        print('-------- getting BGS MAIN targets --------')
        bgs = cuts.isBGS(
            rfiberflux=ff, gflux=gflux, rflux=rflux,
            zflux=zflux, w1flux=w1flux, w2flux=w2flux,
            gnobs=gnobs, rnobs=rnobs, znobs=znobs,
            gfracmasked=gfracmasked, rfracmasked=rfracmasked,
            zfracmasked=zfracmasked, gfracflux=gfracflux,
            rfracflux=rfracflux, zfracflux=zfracflux,
            gfracin=gfracin, rfracin=rfracin, zfracin=zfracin,
            gfluxivar=gfluxivar, rfluxivar=rfluxivar,
            zfluxivar=zfluxivar, maskbits=maskbits,
            Grr=Grr, refcat=refcat, w1snr=w1snr, gaiagmag=gaiagmag,
            primary=prim, targtype=targtype)
    elif survey == 'sv':
        print('-------- getting BGS SV targets --------')
        bgs = sv1_cuts.isBGS(
            gflux=gflux, rflux=rflux, zflux=zflux, w1flux=w1flux, w2flux=w2flux,
            rfiberflux=ff, gnobs=gnobs, rnobs=rnobs, znobs=znobs,
            gfracmasked=gfracmasked, rfracmasked=rfracmasked, zfracmasked=zfracmasked,
            gfracflux=gfracflux, rfracflux=rfracflux, zfracflux=zfracflux,
            gfracin=gfracin, rfracin=rfracin, zfracin=zfracin,
            gfluxivar=gfluxivar, rfluxivar=rfluxivar, zfluxivar=zfluxivar,
            maskbits=maskbits, Grr=Grr, w1snr=w1snr, gaiagmag=gaiagmag,
            objtype=objtype, primary=prim, targtype=targtype)
    else: raise ValueError('%s is not a valid survey programme.' %(survey))
                    
    return bgs

## BGS assess comparison DESITARGET with MYCODE for DR8

### MAIN

In [3]:
#run my bgs script on one sweep brick
mastercat = '/global/project/projectdirs/cosmo/data/legacysurvey/dr8/south/sweep/8.0/sweep-190p000-200p005.fits'
cat = getBGSbits(mycatpath=mastercat, outdir=None, mycat=True, getmycat=True)

---- BGSMASK key: ---- 
	 BS, 0, 1
	 MS, 1, 2
	 GC, 2, 4
	 LG, 3, 8
	 allmask, 4, 16
	 nobs, 5, 32
	 SG, 6, 64
	 SGSV, 7, 128
	 FMC, 8, 256
	 FMC2, 9, 512
	 CC, 10, 1024
	 QC_FM, 11, 2048
	 QC_FI, 12, 4096
	 QC_FF, 13, 8192
	 QC_FM2, 14, 16384
	 QC_FI2, 15, 32768
	 QC_FF2, 16, 65536
	 QC_IVAR, 17, 131072
	 bgs_any, 20, 1048576
	 bgs_bright, 21, 2097152
	 bgs_faint, 22, 4194304
	 bgs_sv_any, 30, 1073741824
	 bgs_sv_bright, 31, 2147483648
	 bgs_sv_faint, 32, 4294967296
	 bgs_sv_faint_ext, 33, 8589934592
	 bgs_sv_fibmag, 34, 17179869184
	 bgs_sv_lowq, 35, 34359738368
	 bgs_sv_any_wqc, 36, 68719476736
	 bgs_sv_lowq_wqc, 37, 137438953472
---- Sanity Check ---- 
	 BS, 3877196, 3877196
	 MS, 3833058, 3833058
	 GC, 3949441, 3949441
	 LG, 3944628, 3944628
	 allmask, 3938037, 3938037
	 nobs, 3948722, 3948722
	 SG, 3766688, 3766688
	 SGSV, 3767362, 3767362
	 FMC, 3934112, 3934112
	 FMC2, 3934643, 3934643
	 CC, 3776143, 3776143
	 QC_FM, 3916707, 3916707
	 QC_FI, 3947746, 3947746
	 QC_FF, 3927141, 

In [4]:
#run desitarget on the same sweep brick
#see: /global/homes/q/qmxp55/DESI/bgs_main/select_targets_bgs.py
#cat1 = fitsio.read('/global/cscratch1/sd/qmxp55/desitarget_output/test_lslga_1.fits/targets/main/resolve/targets-drX-hp-X.fits')
cat1b = runtest(rff=True, targtype='bright', primary1=False, file=mastercat)
cat1f = runtest(rff=True, targtype='faint', primary1=False, file=mastercat)

-------- getting BGS MAIN targets --------
-------- getting BGS MAIN targets --------


In [7]:
#bgsf1 = (cat1f['BGS_TARGET'] & 2**(0)) != 0
#bgsb1 = (cat1b['BGS_TARGET'] & 2**(1)) != 0

bgsb = ((cat['BGSBITS'] & 2**(21)) != 0)
bgsf = ((cat['BGSBITS'] & 2**(22)) != 0)
bgs = ((cat['BGSBITS'] & 2**(20)) != 0)

In [8]:
print('Targ. Dens. of BGS faint: %.3f (DESITARGET) \t %.3f (MYCODE)' %(np.sum(cat1f)/47, np.sum(bgsf)/47))
print('Targ. Dens. of BGS bright: %.3f (DESITARGET) \t %.3f (MYCODE)' %(np.sum(cat1b)/47, np.sum(bgsb)/47))

Targ. Dens. of BGS faint: 570.064 (DESITARGET) 	 568.298 (MYCODE)
Targ. Dens. of BGS bright: 819.234 (DESITARGET) 	 817.106 (MYCODE)


In [10]:
def bsleak(cat):
    #are we leaking near BS LSLGA galaxies?
    BS = (cat['MASKBITS'] & 2**1)!=0
    obs = (BS)
    print('N:','\t',np.sum(obs))
    print('--------')
    
    for i in np.where(obs)[0]:
        if 'RMAG' not in cat.dtype.names:
            rmag = flux_to_mag(cat['FLUX_R']/cat['MW_TRANSMISSION_R'])
            print('%s \t %s \t %.3f' %(cat['REF_CAT'][i], cat['TYPE'][i], rmag[i]))
        else:
            print('%s \t %s \t %.3f' %(cat['REF_CAT'][i], cat['TYPE'][i], cat['RMAG'][i]))


In [10]:
cat1 = Table.read(mastercat)

In [11]:
#Do we miss the LSLGA in BGS?
LXmy = cat['REF_CAT'] == 'L2'
LXdt = cat1['REF_CAT'] == 'L2'

print('#LSLGA in whole SWEEP: \t %i' %(np.sum(LXdt)))
print('#LSLGA in BGS from MYCAT: \t %i' %(np.sum((bgs) & (LXmy))))
print('#LSLGA in BGS from DESITARGET: \t %i' %(np.sum(((cat1b) | (cat1f)) & (LXdt))))


#LSLGA in whole SWEEP: 	 1275
#LSLGA in BGS from MYCAT: 	 1260
#LSLGA in BGS from DESITARGET: 	 1260


In [12]:
bsleak(cat[bgs])

N: 	 0
--------


In [13]:
bsleak(cat1[(cat1b) | (cat1f)])

N: 	 0
--------


### SV

In [28]:
catDT = Table.read(mastercat)
BSMY = (cat['MASKBITS'] & 2**1)!=0
BSDT = (catDT['MASKBITS'] & 2**1)!=0

In [15]:
keys = ["bright", "faint", "faint_ext", "lowq", "fibmag"]
#keys = ["lowq"]
for key in keys:
    SV_MY = ((cat['BGSBITS'] & 2**(bgsmask()['bgs_sv_'+key])) != 0)
    SV_DT = runtest(rff=True, targtype=key, primary1=False, file=mastercat, survey='sv')
    print('Targ. Dens. of BGS SV %s: \t %.3f (DESITARGET) \t %.3f (MYCODE)' %(key, np.sum(SV_DT)/47, np.sum(SV_MY)/47))
    print('galaxies in bright stars bit: \t %.3f (DESITARGET) \t %.3f (MYCODE)' %(np.sum((SV_DT) & (BSDT))/47, np.sum((SV_MY) & (BSMY))/47))

-------- getting BGS SV targets --------
Targ. Dens. of BGS SV bright: 	 823.149 (DESITARGET) 	 836.468 (MYCODE)
galaxies in bright stars bit: 	 0.000 (DESITARGET) 	 0.000 (MYCODE)
-------- getting BGS SV targets --------
Targ. Dens. of BGS SV faint: 	 723.404 (DESITARGET) 	 733.936 (MYCODE)
galaxies in bright stars bit: 	 0.000 (DESITARGET) 	 0.000 (MYCODE)
-------- getting BGS SV targets --------
Targ. Dens. of BGS SV faint_ext: 	 630.298 (DESITARGET) 	 637.617 (MYCODE)
galaxies in bright stars bit: 	 0.000 (DESITARGET) 	 0.000 (MYCODE)
-------- getting BGS SV targets --------
Targ. Dens. of BGS SV lowq: 	 30.596 (DESITARGET) 	 6.745 (MYCODE)
galaxies in bright stars bit: 	 0.000 (DESITARGET) 	 0.000 (MYCODE)
-------- getting BGS SV targets --------
Targ. Dens. of BGS SV fibmag: 	 181.766 (DESITARGET) 	 183.043 (MYCODE)
galaxies in bright stars bit: 	 0.000 (DESITARGET) 	 0.000 (MYCODE)


## BGS assess comparison DESITARGET with MYCODE for DR9

In [33]:
#run my bgsconcatenate on one sweep brick
mastercat = '/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/south/sweep/9.0/sweep-200p005-210p010.fits'
cat = getBGSbits(mycatpath=mastercat, outdir=None, mycat=True, getmycat=True)

---- BGSMASK key: ---- 
	 BS, 0, 1
	 MS, 1, 2
	 GC, 2, 4
	 LG, 3, 8
	 allmask, 4, 16
	 nobs, 5, 32
	 SG, 6, 64
	 SGSV, 7, 128
	 FMC, 8, 256
	 FMC2, 9, 512
	 CC, 10, 1024
	 QC_FM, 11, 2048
	 QC_FI, 12, 4096
	 QC_FF, 13, 8192
	 QC_FM2, 14, 16384
	 QC_FI2, 15, 32768
	 QC_FF2, 16, 65536
	 QC_IVAR, 17, 131072
	 bgs_any, 20, 1048576
	 bgs_bright, 21, 2097152
	 bgs_faint, 22, 4194304
	 bgs_sv_any, 30, 1073741824
	 bgs_sv_bright, 31, 2147483648
	 bgs_sv_faint, 32, 4294967296
	 bgs_sv_faint_ext, 33, 8589934592
	 bgs_sv_fibmag, 34, 17179869184
	 bgs_sv_lowq, 35, 34359738368
---- Sanity Check ---- 
	 BS, 4122443, 4122443
	 MS, 4029139, 4029139
	 GC, 4143328, 4143328
	 LG, 4132116, 4132116
	 allmask, 4131173, 4131173
	 nobs, 4142454, 4142454
	 SG, 3973822, 3973822
	 SGSV, 3975413, 3975413
	 FMC, 4128867, 4128867
	 FMC2, 4129521, 4129521
	 CC, 3951355, 3951355
	 QC_FM, 4116211, 4116211
	 QC_FI, 4141585, 4141585
	 QC_FF, 4030544, 4030544
	 QC_FM2, 4133878, 4133878
	 QC_FI2, 4142772, 4142772
	 QC_FF2

In [21]:
#run desitarget on the same sweep brick
#see: /global/homes/q/qmxp55/DESI/bgs_main/select_targets_bgs.py
#cat1 = fitsio.read('/global/cscratch1/sd/qmxp55/desitarget_output/test_lslga_1.fits/targets/main/resolve/targets-drX-hp-X.fits')
cat1b = runtest(rff=True, targtype='bright', primary1=False, file=mastercat)
cat1f = runtest(rff=True, targtype='faint', primary1=False, file=mastercat)

-------- getting BGS MAIN targets --------
-------- getting BGS MAIN targets --------


In [26]:
#bgsf1 = (cat1f['BGS_TARGET'] & 2**(0)) != 0
#bgsb1 = (cat1b['BGS_TARGET'] & 2**(1)) != 0

bgsb = ((cat['BGSBITS'] & 2**(21)) != 0)
bgsf = ((cat['BGSBITS'] & 2**(22)) != 0)
bgs = ((cat['BGSBITS'] & 2**(20)) != 0)

In [6]:
print('Targ. Dens. of BGS faint: %.3f (DESITARGET) \t %.3f (MYCODE)' %(np.sum(cat1f)/47, np.sum(bgsf)/47))
print('Targ. Dens. of BGS bright: %.3f (DESITARGET) \t %.3f (MYCODE)' %(np.sum(cat1b)/47, np.sum(bgsb)/47))

Targ. Dens. of BGS faint: 591.660 (DESITARGET) 	 591.660 (MYCODE)
Targ. Dens. of BGS bright: 870.894 (DESITARGET) 	 870.894 (MYCODE)


In [7]:
cat1 = Table.read(mastercat)

In [8]:
#Do we miss the LSLGA in BGS?
LXmy = cat['REF_CAT'] == 'L4'
LXdt = cat1['REF_CAT'] == 'L4'

print('#LSLGA in whole SWEEP: \t %i' %(np.sum(LXdt)))
print('#LSLGA in BGS from MYCAT: \t %i' %(np.sum((bgs) & (LXmy))))
print('#LSLGA in BGS from DESITARGET: \t %i' %(np.sum(((cat1b) | (cat1f)) & (LXdt))))


#LSLGA in whole SWEEP: 	 0
#LSLGA in BGS from MYCAT: 	 0
#LSLGA in BGS from DESITARGET: 	 0


In [11]:
bsleak(cat[bgs])

N: 	 0
--------


In [12]:
bsleak(cat1[(cat1b) | (cat1f)])

N: 	 0
--------


### SV

In [34]:
catDT = Table.read(mastercat)
BSMY = (cat['MASKBITS'] & 2**1)!=0
BSDT = (catDT['MASKBITS'] & 2**1)!=0

In [36]:
#keys = ["bright", "faint", "faint_ext", "lowq", "fibmag"]
keys = ["lowq", "fibmag"]
for key in keys:
    SV_MY = ((cat['BGSBITS'] & 2**(bgsmask()['bgs_sv_'+key])) != 0)
    SV_DT = runtest(rff=True, targtype=key, primary1=False, file=mastercat, survey='sv')
    print('Targ. Dens. of BGS SV %s: \t %.3f (DESITARGET) \t %.3f (MYCODE)' %(key, np.sum(SV_DT)/47, np.sum(SV_MY)/47))
    print('galaxies in bright stars bit: \t %.3f (DESITARGET) \t %.3f (MYCODE)' %(np.sum((SV_DT) & (BSDT))/47, np.sum((SV_MY) & (BSMY))/47))

-------- getting BGS SV targets --------
Targ. Dens. of BGS SV lowq: 	 28.745 (DESITARGET) 	 28.745 (MYCODE)
galaxies in bright stars bit: 	 0.000 (DESITARGET) 	 0.000 (MYCODE)
-------- getting BGS SV targets --------
Targ. Dens. of BGS SV fibmag: 	 169.383 (DESITARGET) 	 169.383 (MYCODE)
galaxies in bright stars bit: 	 0.000 (DESITARGET) 	 0.000 (MYCODE)
